In [76]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tqdm import tqdm

from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor

from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import r2_score as r2
from sklearn.metrics import mean_squared_log_error as msle

In [6]:
train = pd.read_csv('/Users/garrisonmond/Desktop/Binance Bot/Equador/train.csv')
test = pd.read_csv('/Users/garrisonmond/Desktop/Binance Bot/Equador/test.csv')
oil = pd.read_csv('/Users/garrisonmond/Desktop/Binance Bot/Equador/oil.csv')
holidays = pd.read_csv("/Users/garrisonmond/Desktop/Binance Bot/Equador/holidays_events.csv")
stores = pd.read_csv("/Users/garrisonmond/Desktop/Binance Bot/Equador/stores.csv")
transactions = pd.read_csv("/Users/garrisonmond/Desktop/Binance Bot/Equador/transactions.csv")

In [7]:
for i in tqdm(range(len(holidays))):
    if holidays.transferred[i]:
        temp = holidays.description[i]
        temp2 = 'Traslado ' + temp
        holidays.loc[i, 'description'] = 0
        holidays.loc[i, 'type'] = 0
        holidays.loc[i, 'locale'] = 0
        holidays.loc[i, 'locale_name'] = 0
        holidays.loc[i, 'transferred'] = False
        for j in range(i+1, len(holidays)):
            if holidays.description[j] == temp2:
                holidays.loc[j, 'description'] = temp
                break

100%|██████████████████████████████████████| 350/350 [00:00<00:00, 27367.76it/s]


In [8]:
test = test.merge(oil, on = 'date', how = 'left')
train = train.merge(oil, on = 'date', how = 'left')
test = test.merge(stores, on = 'store_nbr', how = 'left')
train = train.merge(stores, on = 'store_nbr', how = 'left')
train = train.merge(holidays, on = 'date', how = 'left')
test = test.merge(holidays, on = 'date', how = 'left')
train = train.merge(transactions, on = ['date', 'store_nbr'], how = 'left')
test = test.merge(transactions, on = ['date', 'store_nbr'], how = 'left')

In [9]:
Id = test.id

In [10]:
train.drop('id', axis = 1, inplace = True)
test.drop('id', axis = 1, inplace = True)

In [11]:
day = []
year = []
month = []
for i in train.date:
  year.append(int(i[:4]))
  month.append(int(i[5:7]))
  day.append(int(i[8:]))
train['day'] = day
train['year'] = year
train['month'] = month

In [12]:
day = []
year = []
month = []
for i in test.date:
  year.append(int(i[:4]))
  month.append(int(i[5:7]))
  day.append(int(i[8:]))
test['day'] = day
test['year'] = year
test['month'] = month

In [13]:
train.drop('date', axis = 1, inplace = True)
test.drop('date', axis = 1, inplace = True)

In [14]:
train.drop('transferred', axis = 1, inplace = True)
test.drop('transferred', axis = 1, inplace = True)

In [15]:
train.drop('transactions', axis = 1, inplace = True)

In [16]:
wage = []
for i in tqdm(range(len(train))):
    if train.month[i] in [1, 3, 5, 7, 8, 10, 12] and train.day[i] == 31:
        wage.append(1)
    elif train.month[i] in [4, 6, 9, 11] and train.day[i] == 30:
        wage.append(1)
    elif train.month[i] == 2 and train.year[i] % 4 == 0 and train.day[i] == 29:
        wage.append(1)
    elif train.month[i] == 2 and train.year[i] % 4 != 0 and train.day[i] == 28:
        wage.append(1)
    else:
        wage.append(0)
train['wage'] = wage

100%|██████████████████████████████| 3054348/3054348 [02:39<00:00, 19163.23it/s]


In [17]:
wage = []
for i in tqdm(range(len(test))):
    if test.month[i] in [1, 3, 5, 7, 8, 10, 12] and test.day[i] == 31:
        wage.append(1)
    elif test.month[i] in [4, 6, 9, 11] and test.day[i] == 30:
        wage.append(1)
    elif test.month[i] == 2 and test.year[i] % 4 == 0 and test.day[i] == 29:
        wage.append(1)
    elif test.month[i] == 2 and test.year[i] % 4 != 0 and test.day[i] == 28:
        wage.append(1)
    else:
        wage.append(0)
test['wage'] = wage

100%|██████████████████████████████████| 28512/28512 [00:00<00:00, 38710.46it/s]


In [18]:
def label_encoder(df):
    arr2 = []
    for k in df.columns:
        arr1 = []
        temp = df[k].unique()
        t1 = len(df[k])
        t2 = len(temp)
        for i in tqdm(range(t1)):
            if df[k][i]==0:
                arr1.append(0)
            for j in range(t2):
                if temp[j] == df[k][i]:
                    arr1.append(j+1)
        arr2.append(arr1)
    return pd.DataFrame(arr2).transpose()

In [118]:
columns = ['family', 'city', 'state', 'type_x', 'type_y', 'locale','locale_name', 'description']                        

In [119]:
train[columns] = train[columns].fillna(0)
test[columns] = test[columns].fillna(0)

/var/folders/fq/35bmctzd41ndf6nxlgkwb4_m0000gn/T/ipykernel_56848/3738096166.py:2: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  test[columns] = test[columns].fillna(0)


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [48]:
test[columns] = label_encoder(test[columns])

100%|██████████████████████████████████| 28512/28512 [00:00<00:00, 99812.70it/s]


In [49]:
train[columns] = label_encoder(train[columns])

100%|███████████████████████████████| 3054348/3054348 [40:48<00:00, 1247.30it/s]


In [50]:
for i in train.columns:
    train[i] = train[i].fillna(train[i].mean())

In [51]:
for i in test.columns:
    test[i] = test[i].fillna(test[i].mean())

In [120]:
y = train.sales

In [121]:
X = train.drop('sales', axis = 1)

In [110]:
scaler = StandardScaler()

In [107]:
#for i in tqdm(X.family.unique()):
#    scaler = scaler.fit(X[X.family == i][:int(len(X[X.family == i])*0.8)])
#    X[X.family == i][:int(len(X[X.family == i])*0.8)] = scaler.transform(X[X.family == i][:int(len(X[X.family == i])*0.8)])
#    X[X.family == i][int(len(X[X.family == i])*0.8)+1:] = scaler.transform(X[X.family == i][int(len(X[X.family == i])*0.8)+1:])

  0%|                                                    | 0/33 [00:00<?, ?it/s]/var/folders/fq/35bmctzd41ndf6nxlgkwb4_m0000gn/T/ipykernel_56848/2449011035.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[X.family == i][:int(len(X[X.family == i])*0.8)] = scaler.transform(X[X.family == i][:int(len(X[X.family == i])*0.8)])
/Users/garrisonmond/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1835: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value[:, i].t

/var/folders/fq/35bmctzd41ndf6nxlgkwb4_m0000gn/T/ipykernel_56848/2449011035.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[X.family == i][int(len(X[X.family == i])*0.8)+1:] = scaler.transform(X[X.family == i][int(len(X[X.family == i])*0.8)+1:])
/Users/garrisonmond/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1835: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value[:, i].tolist(), pi)
 15%|██████▋                                     | 5/33 [00:02<

 27%|████████████                                | 9/33 [00:04<00:11,  2.11it/s]/var/folders/fq/35bmctzd41ndf6nxlgkwb4_m0000gn/T/ipykernel_56848/2449011035.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[X.family == i][:int(len(X[X.family == i])*0.8)] = scaler.transform(X[X.family == i][:int(len(X[X.family == i])*0.8)])
/Users/garrisonmond/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1835: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value[:, i].t

/var/folders/fq/35bmctzd41ndf6nxlgkwb4_m0000gn/T/ipykernel_56848/2449011035.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[X.family == i][int(len(X[X.family == i])*0.8)+1:] = scaler.transform(X[X.family == i][int(len(X[X.family == i])*0.8)+1:])
/Users/garrisonmond/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1835: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value[:, i].tolist(), pi)
 42%|██████████████████▏                        | 14/33 [00:06<

 55%|███████████████████████▍                   | 18/33 [00:08<00:06,  2.16it/s]/var/folders/fq/35bmctzd41ndf6nxlgkwb4_m0000gn/T/ipykernel_56848/2449011035.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[X.family == i][:int(len(X[X.family == i])*0.8)] = scaler.transform(X[X.family == i][:int(len(X[X.family == i])*0.8)])
/Users/garrisonmond/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1835: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value[:, i].t

/var/folders/fq/35bmctzd41ndf6nxlgkwb4_m0000gn/T/ipykernel_56848/2449011035.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[X.family == i][int(len(X[X.family == i])*0.8)+1:] = scaler.transform(X[X.family == i][int(len(X[X.family == i])*0.8)+1:])
/Users/garrisonmond/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1835: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value[:, i].tolist(), pi)
 70%|█████████████████████████████▉             | 23/33 [00:10<

 82%|███████████████████████████████████▏       | 27/33 [00:12<00:02,  2.17it/s]/var/folders/fq/35bmctzd41ndf6nxlgkwb4_m0000gn/T/ipykernel_56848/2449011035.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[X.family == i][:int(len(X[X.family == i])*0.8)] = scaler.transform(X[X.family == i][:int(len(X[X.family == i])*0.8)])
/Users/garrisonmond/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1835: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value[:, i].t

/var/folders/fq/35bmctzd41ndf6nxlgkwb4_m0000gn/T/ipykernel_56848/2449011035.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[X.family == i][int(len(X[X.family == i])*0.8)+1:] = scaler.transform(X[X.family == i][int(len(X[X.family == i])*0.8)+1:])
/Users/garrisonmond/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1835: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value[:, i].tolist(), pi)
 97%|█████████████████████████████████████████▋ | 32/33 [00:15<

In [123]:
X_tr, X_t = X[:int(len(X)*0.8)], X[int(len(X)*0.8)+1:]
y_tr, y_t = y[:int(len(y)*0.8)], y[int(len(y)*0.8)+1:]

In [124]:
scaler = scaler.fit(X_tr)
X_tr = scaler.transform(X_tr)
X_t = scaler.transform(X_t)

In [125]:
def score(pred, true):
    print("MSE: ", mse(true, pred))
    print("R2 score: ", r2(true, pred))

In [126]:
lr = LinearRegression().fit(X_tr, y_tr)
grad = GradientBoostingRegressor().fit(X_tr, y_tr)
rnd = RandomForestRegressor().fit(X_tr, y_tr)

In [127]:
score(lr.predict(X_t), y_t)
score(grad.predict(X_t), y_t)
score(rnd.predict(X_t), y_t)

MSE:  1175150.599984328
R2 score:  0.3696600910979183
MSE:  493234.49091172905
R2 score:  0.7354335826635239
MSE:  238639.55408718847
R2 score:  0.8719959511693584


In [135]:
test = scaler.transform(test)
prd = pd.DataFrame([Id,rnd.predict(test)]).transpose()
prd.columns = ['id','sales']
prd.id = prd.id.astype(int)
prd.set_index('id', inplace =True)
prd.to_csv("RND3.csv")

In [134]:
test

array([[-1.87338555e+00, -1.96182615e+00, -1.70831538e-01, ...,
        -1.86784601e+03, -1.69090498e+00, -1.21523081e+00],
       [-1.87338555e+00, -1.95079671e+00, -1.70831538e-01, ...,
        -1.86784601e+03, -1.69090498e+00, -1.21523081e+00],
       [-1.87338555e+00, -1.93976727e+00, -1.49098869e-01, ...,
        -1.86784601e+03, -1.69090498e+00, -1.21523081e+00],
       ...,
       [-1.84045215e+00, -1.63094294e+00, -1.59965203e-01, ...,
        -1.86784601e+03, -1.69090498e+00,  3.09815530e+01],
       [-1.84045215e+00, -1.61991350e+00, -7.30345255e-02, ...,
        -1.86784601e+03, -1.69090498e+00,  3.09815530e+01],
       [-1.84045215e+00, -1.60888406e+00, -1.70831538e-01, ...,
        -1.86784601e+03, -1.69090498e+00,  3.09815530e+01]])

In [ ]:
for i in [1,5,10,50,100]:
  l = int(len(train)/33)
  for j in range(1, len(train)):
    if l % i == 0:
      break
    l -= 1
  X = pd.DataFrame(np.array(train.drop('sales', axis = 1)[:l]).reshape(int(l / i), len(16) * i)[:-1])
  y = []
  for j in range(int(l / i)):
    y.append(sum(df["Delta"][j * i:(j + 1) * i]))
  y = pd.DataFrame(y[1:])
  y["Sign"] = np.sign(y)
  y["Sign"][y["Sign"] == 0] = 1
  X_tr, X_t = X[:int(len(X) * 0.8)], X[int(len(X) * 0.8) + 1:]
  y_tr, y_t = y["Sign"][:int(len(y) * 0.8)], y["Sign"][int(len(y) * 0.8) + 1:]

  scaler = MinMaxScaler().fit(X_tr)

  X_tr = pd.DataFrame(scaler.transform(X_tr), index=X_tr.index)
  X_t = pd.DataFrame(scaler.transform(X_t), index=X_t.index)

  X = pd.concat([X_tr, X_t])
  y = pd.concat([y_tr, y_t])

  lr = LogisticRegression(max_iter=10000)
  knn = KNeighborsClassifier()
  tree = DecisionTreeClassifier()
  rnd = RandomForestClassifier()

  knn = knn.fit(X_tr, y_tr)
  lr = lr.fit(X_tr, y_tr)
  tree = tree.fit(X_tr, y_tr)
  rnd = rnd.fit(X_tr, y_tr)

  print("LR i = " + str(i))
  temp = score(X, y, lr)
  lr_accuracy[0].append(temp[0])
  lr_accuracy[1].append(temp[1])
  lr_recall[0].append(temp[2])
  lr_recall[1].append(temp[3])
  lr_precision[0].append(temp[4])
  lr_precision[1].append(temp[5])
  lr_f1[0].append(temp[6])
  lr_f1[1].append(temp[7])

  print("KNN i = " + str(i))
  temp = score(X, y, knn)
  knn_accuracy[0].append(temp[0])
  knn_accuracy[1].append(temp[1])
  knn_recall[0].append(temp[2])
  knn_recall[1].append(temp[3])
  knn_precision[0].append(temp[4])
  knn_precision[1].append(temp[5])
  knn_f1[0].append(temp[6])
  knn_f1[1].append(temp[7])  
    
  print("TREE i = " + str(i))
  temp = score(X, y, tree)
  tree_accuracy[0].append(temp[0])
  tree_accuracy[1].append(temp[1])
  tree_recall[0].append(temp[2])
  tree_recall[1].append(temp[3])
  tree_precision[0].append(temp[4])
  tree_precision[1].append(temp[5])
  tree_f1[0].append(temp[6])
  tree_f1[1].append(temp[7])  
    
  print("RND i = " + str(i))
  temp = score(X, y, rnd)
  rnd_accuracy[0].append(temp[0])
  rnd_accuracy[1].append(temp[1])
  rnd_recall[0].append(temp[2])
  rnd_recall[1].append(temp[3])
  rnd_precision[0].append(temp[4])
  rnd_precision[1].append(temp[5])
  rnd_f1[0].append(temp[6])
  rnd_f1[1].append(temp[7])  

In [87]:
##Нормировать по каждой семье отдельно

In [ ]:
##Зависимость от кол-во дней

In [129]:
(13*12)/2*11*(4*3)/2*(4*3)/2*4 # б

5559840.0

In [130]:
13*(12*11/2)*(4*3*2)/6*4*4 # в

54912.0

In [131]:
13*12*(4*3*2)/6*(4*3)/2 # г

3744.0

In [132]:
4*(13*12*11*10*9)/120 # д

5148.0

In [133]:
4*(13*12)/2*11*10*9 # е

308880.0